In [116]:
import os
import pandas as pd
import json
import numpy as np
import random
import gensim
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk import RegexpTokenizer
from gensim.models import Word2Vec
import matplotlib.pyplot as plt

import os
from nltk.corpus import stopwords

stop = stopwords.words('english')
stop+=['?','!','.',',',':',';']

%matplotlib inline
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/williamcottrell72/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/williamcottrell72/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
followers = pd.read_json('https://s3-us-west-1.amazonaws.com/primebotbucket/Data/08_08_18_followers.json', lines=True)

In [5]:
messages1 = pd.read_json('https://s3-us-west-1.amazonaws.com/primebotbucket/Data/08_08_18_messages.json', lines=True)

In [6]:
messages2 = pd.read_json('https://s3-us-west-1.amazonaws.com/primebotbucket/Data/08_08_18_v2messages.json', lines=True)

In [7]:
messages3 = pd.read_json('https://s3-us-west-1.amazonaws.com/primebotbucket/Data/6_25_18_messages.json', lines=True)

In [65]:
m1=messages1['message']
messages=[]
for mes in m1:
    if 'message' in mes.keys():
        try:
            messages.append(mes['message']['text'])
        except:
            pass
    

In [209]:
subset=np.random.choice(range(len(messages1)),5000)
subset2=np.random.choice(range(len(messages1)),100)
short_messages=messages1.iloc[subset]
short_messages2=messages1.iloc[subset2]

In [216]:
os.getcwd()

'/Users/williamcottrell72'

In [208]:
def clean_messages(short_messages):
    short_messages['new_date']=short_messages['date'].apply(lambda x: x['$date']).values
    short_messages['message_id']=short_messages['_id'].apply(lambda x: x['$oid'])

    text=[]
    sender=[]
    timestamp=[]
    recipient=[]
    nlp=[]
    for x in short_messages['message']:    
        sender.append(x['sender']['id'])
        timestamp.append(x['timestamp'])
        recipient.append(x['recipient']['id'])
        if 'postback' in x.keys():
            text.append(x['postback']['title'])
            nlp.append(0)

        else:
            try:
                text.append(x['message']['text'])
                nlp.append(1)
            except:
                text.append('NaN')
                nlp.append(0)

    replace=pd.DataFrame({'text':text,'sender':sender,'recipient':recipient,'timestamp':timestamp,'nlp':nlp})

    sm=short_messages.reset_index().drop(['_id','message','date'],axis=1)
    sm_new=sm.join(replace)
    return sm_new


In [211]:
res=clean_messages(short_messages2)

/Users/williamcottrell72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/williamcottrell72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [215]:
res[res.nlp==1]

,index,__v,pageId,new_date,message_id,text,sender,recipient,timestamp,nlp
15,103718,0,1.748597e+15,2018-07-27T08:42:41.317Z,5b5adb010caa170014b239c9,HAHAHAHAHA,1801476999944417,1748596732049183,1.532681e+12,1
31,250941,0,1.748597e+15,2018-07-29T02:22:32.828Z,5b5d24e85d30c600141933c5,fuck yiu,1316082558492369,1748596732049183,1.532831e+12,1
36,272599,0,1.748597e+15,2018-07-29T08:50:55.618Z,5b5d7fef5d30c6001419e79a,More pick up lines,1726715647445499,1748596732049183,1.532854e+12,1
46,24930,0,1.748597e+15,2018-07-26T11:28:18.339Z,5b59b052d86a92001459c699,Up your pickup game,1748717728505426,1748596732049183,1.532604e+12,1
51,205044,0,1.748597e+15,2018-07-28T13:00:34.045Z,5b5c68f25d30c6001417b26f,Im sorry,1718344218211401,1748596732049183,1.532783e+12,1
59,33988,0,1.748597e+15,2018-07-26T12:01:45.643Z,5b59b829d86a9200145a0e05,More,1597960256910477,1748596732049183,1.532607e+12,1
61,809565,0,1.553195e+14,2018-08-05T22:11:23.112Z,5b67760b0051520014a5c06c,"YES, It Sure Does!",1729137037134169,155319498535430,1.533507e+12,1
65,108580,0,1.748597e+15,2018-07-27T10:07:04.377Z,5b5aeec80caa170014b2618f,Fj,1334600366642632,1748596732049183,1.532686e+12,1
68,274839,0,1.748597e+15,2018-07-29T09:26:49.609Z,5b5d88595d30c6001419f9e2,Funny T-Shirts,1815624365152088,1748596732049183,1.532856e+12,1
80,45935,0,1.748597e+15,2018-07-26T13:25:27.708Z,5b59cbc70caa170014b0621f,Funny T-shirts,2242475405793416,1748596732049183,1.532612e+12,1
